In [0]:
%sql
/* 04_gold_claim_anomalies.ipynb  
GOAL 1: Approval rates by specialty  
GOAL 2: Denial breakdown by specialty & diagnosis  
GOAL 3: Top high‑cost procedures  
GOAL 4: Rapid‑fire patient claim bursts  

SOURCE: kardia_silver.silver_claims_enriched  
OUTPUT: TABLE: gold_claim_approval_by_specialty
        TABLE: gold_claim_denial_breakdown
        TABLE: gold_high_cost_procedures
        TABLE: gold_rapid_fire_claims

TRIGGER: Full snapshot overwrite each run — simple for small datasets. */

USE kardia_gold;

-- Config
SET kflow.rapid_fire.threshold = 5;

In [0]:
%sql  
-- Approval rates by specialty  
CREATE OR REPLACE TABLE gold_claim_approval_by_specialty AS
SELECT
  COALESCE(provider_specialty, 'Unknown') AS provider_specialty,
  COUNT(*)                                 AS total_claims,
  SUM(CASE WHEN claim_status = 'Approved' THEN 1 ELSE 0 END) AS approved_count,
  ROUND(SUM(CASE WHEN claim_status = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 4) AS approval_rate
FROM kardia_silver.silver_claims_enriched
GROUP BY COALESCE(provider_specialty, 'Unknown');

In [0]:
%sql  
-- Denial breakdown by specialty
CREATE OR REPLACE TABLE gold_claim_denial_breakdown AS
WITH d AS (
  SELECT
    COALESCE(provider_specialty, 'Unknown') AS provider_specialty,
    diagnosis_code,
    COUNT(*) AS denial_count
  FROM kardia_silver.silver_claims_enriched
  WHERE claim_status = 'Denied'
  GROUP BY COALESCE(provider_specialty, 'Unknown'), diagnosis_code
)
SELECT
  provider_specialty,
  diagnosis_code,
  denial_count,
  ROUND(
    denial_count / SUM(denial_count) OVER (PARTITION BY provider_specialty),
    4
  ) AS denial_pct
FROM d;

In [0]:
%sql  
-- Top 10 high‑cost procedures by avg claim amount  
CREATE OR REPLACE TABLE gold_high_cost_procedures AS
SELECT
  procedure_code,
  ROUND(AVG(claim_amount), 2) AS avg_claim_amount,
  COUNT(*)                    AS procedure_count
FROM kardia_silver.silver_claims_enriched
GROUP BY procedure_code
ORDER BY avg_claim_amount DESC
LIMIT 10;

In [0]:
%sql  
-- Rapid‑fire patient claim bursts (>5 claims/day)  
CREATE OR REPLACE TABLE gold_rapid_fire_claims AS
WITH daily_counts AS (
  SELECT
    patient_id,
    CAST(claim_date AS DATE) AS claim_day,
    COUNT(*)                 AS daily_count
  FROM kardia_silver.silver_claims_enriched
  GROUP BY patient_id, CAST(claim_date AS DATE)
)
SELECT *
FROM daily_counts
WHERE daily_count > ${kflow.rapid_fire.threshold};

In [0]:
%sql  
-- Preview: approval rates by specialty  
SELECT *
FROM gold_claim_approval_by_specialty
ORDER BY approval_rate DESC
LIMIT 10;

In [0]:
%sql  
-- Preview: denial breakdown  
SELECT * FROM gold_claim_denial_breakdown LIMIT 10;

In [0]:
%sql  
-- Preview: high‑cost procedures  
SELECT * FROM gold_high_cost_procedures LIMIT 10;

In [0]:
%sql  
-- Preview: rapid‑fire claims  
SELECT * FROM gold_rapid_fire_claims LIMIT 10;